In [ ]:
#Generic Embeddings Evaluator
#Embeddings like doc2vec, BERT, distilBERT were tested using this 


#Upload case docs embedding
import pandas as pd
import numpy as np
from google.colab import files
import io
import ast #string to list
import json
uploaded = files.upload()
doc_embeddings=np.load("case_docs_200_embeddings.npy")
print(doc_embeddings.shape)





Saving case_docs_200_embeddings.npy to case_docs_200_embeddings.npy
(200, 768)


In [ ]:
#Upload Query Embeddings

uploaded3 = files.upload()
query_embeddings=np.load("query_embeddings.npy")
print(query_embeddings.shape)


Saving query_embeddings.npy to query_embeddings.npy
(50, 768)


In [ ]:
#Uploading query.csv ( for relevant list for each query that's given to us)
uploaded2 = files.upload()
qdf= pd.read_csv(io.BytesIO(uploaded2['embedded_queries.csv']))
qdf['relevant'] = qdf['relevant'].apply(ast.literal_eval) #string to list
print(qdf)

Saving embedded_queries.csv to embedded_queries.csv
    Unnamed: 0  ...                                           relevant
0            0  ...                                            [9, 14]
1            1  ...                                           [27, 22]
2            2  ...                                                [1]
3            3  ...                                              [182]
4            4  ...                            [36, 144, 54, 121, 155]
5            5  ...                                  [152, 99, 26, 19]
6            6  ...                                              [130]
7            7  ...                                      [32, 60, 125]
8            8  ...                                           [42, 90]
9            9  ...                                     [185, 180, 86]
10          10  ...                                         [131, 132]
11          11  ...                                                [8]
12          12  ...      

In [ ]:
import json
#Calculating top 10 rel docs
from sklearn.metrics.pairwise import cosine_similarity
N=768 #1024 #number of dimensions for the embedding #1024
k=30
D=200
Q=50
ans=[]
for i,rowq in qdf.iterrows():
  print("Finding for query ",i+1)
  embed_q=query_embeddings[i]
  cosines=[]
  for j in range(D):
    embed_d=doc_embeddings[j]
    embed_q=np.reshape(embed_q,(1,N))
    embed_d=np.reshape(embed_d,(1,N))
    cs=cosine_similarity(embed_d,embed_q)
    cosines.append([cs,j])
  cosines=sorted(cosines,key=lambda x:x[0],reverse=True)[:k] #top k docs sorted by cosine similarity
  top_k_docs=[doc_idx for cs,doc_idx in cosines] #store only the doc indices
  given=rowq["relevant"]
  matches=0
  for ele in top_k_docs:
    if(ele in given):
      matches+=1
  precision_at_k=round(matches/k,2)
  recall_at_k=round(matches/len(given),2)
  ans.append({"index":"AILA_Q"+str(i+1),"top_k":top_k_docs,"relevant":given,"precision@k":precision_at_k,"recall@k":recall_at_k})

ans_df=pd.DataFrame(ans)
print(ans_df)
ans_df.to_csv('ans.csv') 
files.download('ans.csv')

ap=ans_df["precision@k"].sum()/Q
ar=ans_df["recall@k"].sum()/Q
print("The Avg. precision@k for all queries— ",ap)
print("The Avg. recall@k for all queries— ",ar)
f_score=(2*ap*ar)/ (ap+ar)
print("The F-score is ",f_score)


Finding for query  1
Finding for query  2
Finding for query  3
Finding for query  4
Finding for query  5
Finding for query  6
Finding for query  7
Finding for query  8
Finding for query  9
Finding for query  10
Finding for query  11
Finding for query  12
Finding for query  13
Finding for query  14
Finding for query  15
Finding for query  16
Finding for query  17
Finding for query  18
Finding for query  19
Finding for query  20
Finding for query  21
Finding for query  22
Finding for query  23
Finding for query  24
Finding for query  25
Finding for query  26
Finding for query  27
Finding for query  28
Finding for query  29
Finding for query  30
Finding for query  31
Finding for query  32
Finding for query  33
Finding for query  34
Finding for query  35
Finding for query  36
Finding for query  37
Finding for query  38
Finding for query  39
Finding for query  40
Finding for query  41
Finding for query  42
Finding for query  43
Finding for query  44
Finding for query  45
Finding for query  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The Avg. precision@k for all queries—  0.02
The Avg. recall@k for all queries—  0.1748
The F-score is  0.03589322381930185
